# Module 6: Intelligent Agents

Build intelligent agents that reason over financial graph data and make informed decisions.

## Learning Objectives
- Create agents with memory and graph reasoning capabilities
- Build tool-based architectures for financial analysis
- Implement complete investment analysis workflows

## Module Structure
1. **Basic Agent** - Memory and simple tools
2. **Graph Reasoning** - Multi-hop analysis patterns
3. **Complete System** - End-to-end financial intelligence

## Setup and Dependencies

First, we'll install the required packages for building intelligent agents with graph database capabilities.

In [ ]:
# Install required packages for intelligent agents
!pip install neo4j pandas matplotlib

## Import Libraries and Create Mock LLM

We'll import the necessary libraries and create a simple mock LLM for demonstration purposes. In production, you would replace this with a real LLM like OpenAI's GPT or Anthropic's Claude.

# Import essential libraries for agent development
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List, Dict, Any
from dataclasses import dataclass, field
from neo4j import GraphDatabase

In [ ]:
# Create a simple mock LLM for demonstrations
# In production, replace this with OpenAI, Anthropic, or other LLM APIs
class MockLLM:
    def generate(self, prompt: str) -> str:
        """Generate responses based on prompt keywords"""
        if 'apple' in prompt.lower():
            return "Apple shows strong fundamentals. Recommended as solid tech investment with moderate risk."
        elif 'risk' in prompt.lower():
            return "Key risks: competition, regulation, supply chain. Overall moderate risk level."
        return "Analysis indicates positive trends with areas requiring monitoring."

# Initialize the mock LLM
llm = MockLLM()
print("✅ Setup complete")

## Neo4j Database Connection

Every notebook needs to establish a connection to Neo4j. We'll load environment variables and create a database driver with proper error handling.

In [ ]:
# Load environment variables for database connection
from dotenv import load_dotenv
load_dotenv()

# Set up Neo4j connection parameters
NEO4J_URI = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME', 'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD', 'password')

# Create Neo4j driver and helper function
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def run_query(query, parameters=None):
    """Execute a Cypher query and return results as a list of dictionaries"""
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

# Test the database connection
try:
    result = run_query("RETURN 'Connected to Neo4j!' as message")
    print(f"✅ {result[0]['message']}")
except Exception as e:
    print(f"❌ Neo4j connection failed: {e}")
    print("Please ensure Neo4j is running and credentials are correct")

In [ ]:
# Enhanced Agent with Graph Reasoning

class GraphReasoningTool:
    def competitive_analysis(self, company_name: str):
        query = """
        MATCH (c:Company {name: $name})
        OPTIONAL MATCH (c)-[:COMPETES_WITH]->(competitor:Company)
        OPTIONAL MATCH (c)-[:FACES_RISK]->(risk:RiskFactor)<-[:FACES_RISK]-(peer:Company)
        WHERE peer <> c
        RETURN c.name as company, c.sector as sector,
               collect(DISTINCT competitor.name) as direct_competitors,
               collect(DISTINCT peer.name) as risk_peers
        """
        results = run_query(query, {'name': company_name})
        
        if results:
            data = results[0]
            competitors = [c for c in data['direct_competitors'] if c]
            peers = [p for p in data['risk_peers'] if p]
            
            return {
                'success': True,
                'company': data['company'],
                'sector': data['sector'],
                'direct_competitors': competitors,
                'risk_peers': peers,
                'competitive_intensity': 'High' if len(competitors) > 2 else 'Medium' if len(competitors) > 0 else 'Low'
            }
        return {'success': False, 'error': f'Company {company_name} not found'}
    
    def risk_analysis(self, company_name: str):
        query = """
        MATCH (c:Company {name: $name})
        OPTIONAL MATCH (c)-[:FACES_RISK]->(risk:RiskFactor)
        OPTIONAL MATCH (c)-[:FACES_RISK]->(shared_risk:RiskFactor)<-[:FACES_RISK]-(other:Company)
        WHERE other <> c
        RETURN c.name as company,
               collect(DISTINCT risk.name) as direct_risks,
               collect(DISTINCT {risk: shared_risk.name, shared_with: other.name}) as network_risks
        """
        results = run_query(query, {'name': company_name})
        
        if results:
            data = results[0]
            direct_risks = data['direct_risks']
            network_risks = [nr for nr in data['network_risks'] if nr['risk']]
            
            risk_score = len(direct_risks) * 20 + len(network_risks) * 10
            risk_level = 'High' if risk_score >= 60 else 'Medium' if risk_score >= 30 else 'Low'
            
            return {
                'success': True,
                'company': data['company'],
                'direct_risks': direct_risks,
                'network_risks': network_risks,
                'risk_score': min(risk_score, 100),
                'risk_level': risk_level
            }
        return {'success': False, 'error': f'Company {company_name} not found'}

class EnhancedAgent(FinancialAgent):
    def __init__(self, name: str):
        super().__init__(name)
        self.graph_tool = GraphReasoningTool()
    
    def comprehensive_analysis(self, company_name: str):
        # Basic company data
        company_data = self.company_tool.execute(company_name)
        if not company_data['success']:
            return {'error': f'Company {company_name} not found'}
        
        # Graph reasoning
        competitive_analysis = self.graph_tool.competitive_analysis(company_name)
        risk_analysis = self.graph_tool.risk_analysis(company_name)
        
        # Generate comprehensive recommendation
        prompt = f"""
        Investment analysis for {company_name}:
        
        Basic: {company_data['sector']} sector company
        Competition: {competitive_analysis['competitive_intensity']} intensity, {len(competitive_analysis['direct_competitors'])} direct competitors
        Risk: {risk_analysis['risk_level']} risk level, score {risk_analysis['risk_score']}/100
        
        Provide BUY/HOLD/SELL recommendation with reasoning.
        """
        
        recommendation = llm.generate(prompt)
        
        self.memory.add_conversation(f"Full analysis {company_name}", recommendation)
        self.memory.track_entity(company_name, f"Comprehensive analysis")
        self.query_count += 1
        
        return {
            'company': company_name,
            'basic_data': company_data,
            'competitive_analysis': competitive_analysis,
            'risk_analysis': risk_analysis,
            'recommendation': recommendation
        }

# Create enhanced agent
enhanced_agent = EnhancedAgent("GraphAnalyst")
print("✅ Enhanced agent with graph reasoning created")

In [ ]:
# Test Graph Reasoning

print("=== TESTING GRAPH REASONING ===\n")

# Comprehensive analysis
analysis = enhanced_agent.comprehensive_analysis("Apple Inc")

print(f"📊 COMPREHENSIVE ANALYSIS: {analysis['company']}")
print(f"Sector: {analysis['basic_data']['sector']}")
print(f"Competitive Intensity: {analysis['competitive_analysis']['competitive_intensity']}")
print(f"Risk Level: {analysis['risk_analysis']['risk_level']} ({analysis['risk_analysis']['risk_score']}/100)")
print(f"\n💼 Recommendation:\n{analysis['recommendation']}")

print(f"\n📈 Enhanced Agent Status: {enhanced_agent.get_status()}")

In [ ]:
## Lesson 3: Complete Financial Intelligence System

# Complete Financial Intelligence System

class MarketDataTool:
    def __init__(self):
        self.mock_data = {
            'AAPL': {'price': 185.25, 'change_percent': 1.29, 'pe_ratio': 28.5},
            'MSFT': {'price': 415.30, 'change_percent': -0.44, 'pe_ratio': 32.1}
        }
    
    def get_data(self, ticker: str):
        if ticker in self.mock_data:
            return {'success': True, 'data': self.mock_data[ticker]}
        return {'success': False, 'error': f'No data for {ticker}'}

class FinancialIntelligenceSystem:
    def __init__(self):
        self.agent = EnhancedAgent("FinancialIntelligence")
        self.market_tool = MarketDataTool()
        self.portfolio = {}
        self.alerts = []
    
    def analyze_investment(self, company_name: str, ticker: str):
        # Get comprehensive analysis
        analysis = self.agent.comprehensive_analysis(company_name)
        if 'error' in analysis:
            return analysis
        
        # Get market data
        market_data = self.market_tool.get_data(ticker)
        
        # Calculate investment score
        score = 50  # Base score
        
        if market_data['success']:
            data = market_data['data']
            # Price momentum
            if data['change_percent'] > 0:
                score += min(data['change_percent'] * 3, 15)
            # P/E ratio
            if 15 <= data['pe_ratio'] <= 25:
                score += 20
            elif 10 <= data['pe_ratio'] <= 30:
                score += 10
        
        # Risk adjustment
        risk_score = analysis['risk_analysis']['risk_score']
        score -= risk_score * 0.3
        
        # Competitive position
        intensity = analysis['competitive_analysis']['competitive_intensity']
        if intensity == 'Low':
            score += 15
        elif intensity == 'Medium':
            score += 10
        
        investment_score = max(min(score, 100), 0)
        
        return {
            'company': company_name,
            'ticker': ticker,
            'investment_score': investment_score,
            'market_data': market_data.get('data', {}),
            'analysis': analysis,
            'recommendation': 'BUY' if investment_score >= 70 else 'HOLD' if investment_score >= 50 else 'SELL'
        }
    
    def add_to_portfolio(self, company_name: str, ticker: str):
        analysis = self.analyze_investment(company_name, ticker)
        
        self.portfolio[ticker] = {
            'company': company_name,
            'analysis': analysis,
            'added_date': datetime.now()
        }
        
        # Generate alert if low score
        if analysis['investment_score'] < 40:
            self.alerts.append({
                'type': 'low_score',
                'message': f"{company_name} has low investment score: {analysis['investment_score']:.1f}",
                'timestamp': datetime.now()
            })
        
        return analysis
    
    def get_portfolio_summary(self):
        if not self.portfolio:
            return {'message': 'Portfolio is empty'}
        
        scores = [info['analysis']['investment_score'] for info in self.portfolio.values()]
        
        return {
            'total_companies': len(self.portfolio),
            'average_score': sum(scores) / len(scores),
            'companies': [
                {
                    'company': info['company'],
                    'ticker': ticker,
                    'score': info['analysis']['investment_score'],
                    'recommendation': info['analysis']['recommendation']
                }
                for ticker, info in sorted(self.portfolio.items(), 
                                         key=lambda x: x[1]['analysis']['investment_score'], 
                                         reverse=True)
            ]
        }

# Create financial intelligence system
intelligence_system = FinancialIntelligenceSystem()
print("✅ Complete financial intelligence system created")

In [ ]:
# Test Complete System

print("=== TESTING COMPLETE FINANCIAL INTELLIGENCE SYSTEM ===\n")

# Analyze and add companies to portfolio
companies = [("Apple Inc", "AAPL"), ("Microsoft Corp", "MSFT")]

for company, ticker in companies:
    print(f"📊 Analyzing {company} ({ticker})")
    analysis = intelligence_system.add_to_portfolio(company, ticker)
    print(f"   Score: {analysis['investment_score']:.1f}/100")
    print(f"   Recommendation: {analysis['recommendation']}")
    print()

# Portfolio summary
summary = intelligence_system.get_portfolio_summary()
print("📈 PORTFOLIO SUMMARY")
print(f"Companies: {summary['total_companies']}")
print(f"Average Score: {summary['average_score']:.1f}/100")

print("\n🏆 Rankings:")
for i, company in enumerate(summary['companies'], 1):
    print(f"  {i}. {company['company']} ({company['ticker']}): {company['score']:.1f} - {company['recommendation']}")

# System alerts
if intelligence_system.alerts:
    print(f"\n⚠️ Alerts: {len(intelligence_system.alerts)}")
    for alert in intelligence_system.alerts:
        print(f"  • {alert['message']}")

print(f"\n📊 System Stats:")
agent_status = intelligence_system.agent.get_status()
print(f"  Queries: {agent_status['queries']}")
print(f"  Entities Tracked: {agent_status['entities_tracked']}")
print(f"  Portfolio Companies: {len(intelligence_system.portfolio)}")

In [ ]:
# Simple Visualization Dashboard

def create_dashboard():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Portfolio scores
    if intelligence_system.portfolio:
        companies = []
        scores = []
        colors = []
        
        for ticker, info in intelligence_system.portfolio.items():
            companies.append(f"{info['company'][:10]}\n({ticker})")
            score = info['analysis']['investment_score']
            scores.append(score)
            colors.append('green' if score >= 70 else 'orange' if score >= 50 else 'red')
        
        bars = ax1.bar(companies, scores, color=colors)
        ax1.set_title('Portfolio Investment Scores')
        ax1.set_ylabel('Score (0-100)')
        ax1.set_ylim(0, 100)
        
        # Add score labels
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
                    f'{score:.0f}', ha='center', va='bottom', fontweight='bold')
    
    # Agent memory over time
    conversations = intelligence_system.agent.memory.conversations
    if conversations:
        times = [conv['time'] for conv in conversations]
        counts = list(range(1, len(times) + 1))
        
        ax2.plot(times, counts, marker='o', linewidth=2)
        ax2.set_title('Agent Activity Timeline')
        ax2.set_ylabel('Cumulative Conversations')
        ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    print("\n📊 DASHBOARD SUMMARY")
    print("="*40)
    
    if intelligence_system.portfolio:
        summary = intelligence_system.get_portfolio_summary()
        print(f"📈 Portfolio: {summary['total_companies']} companies, avg {summary['average_score']:.1f}")
    
    agent_status = intelligence_system.agent.get_status()
    print(f"🤖 Agent: {agent_status['queries']} queries, {agent_status['entities_tracked']} entities")
    print(f"⚠️ Alerts: {len(intelligence_system.alerts)}")

create_dashboard()

In [ ]:
## Module Summary

🎉 **Module 6: Agents - Complete!**

### What You Built
✅ **Basic Agent** - Memory system and tool integration  
✅ **Graph Reasoning** - Multi-hop analysis over financial networks  
✅ **Complete System** - End-to-end investment analysis with portfolio management

### Key Capabilities
- Agent memory for conversation and entity tracking
- Graph-based reasoning for competitive and risk analysis  
- Tool integration for external data sources
- Investment scoring and portfolio monitoring
- Alert generation and performance tracking

### Next Steps
🚀 **Extensions to Try:**
- Connect to real financial APIs (Alpha Vantage, SEC EDGAR)
- Add more sophisticated reasoning patterns
- Implement real-time monitoring and alerts
- Build web interface with Streamlit

📚 **Continue Learning:**
- Neo4j Graph Data Science Library
- Advanced agent frameworks (LangChain, CrewAI)
- Production deployment patterns

### Course Complete! 🏆
You've mastered all 6 modules of **Modular GenAI Workshops 2025**:
1. Graph Basics → 2. Structured Data → 3. Unstructured Data → 4. Analytics → 5. Retrievers → 6. **Agents**

**You're now equipped to build sophisticated graph-powered AI systems!**

# Final System Status

print("🎊 MODULE 6: AGENTS - COMPLETED! 🎊")
print("🏆 MODULAR GENAI WORKSHOPS 2025 - COURSE COMPLETED! 🏆\n")

# Final stats
agent_status = intelligence_system.agent.get_status()
portfolio_summary = intelligence_system.get_portfolio_summary()

print("📊 FINAL SYSTEM STATISTICS")
print("="*40)
print(f"🤖 Agent Queries: {agent_status['queries']}")
print(f"🧠 Conversations: {agent_status['conversations']}")
print(f"🏢 Entities Tracked: {agent_status['entities_tracked']}")

if 'total_companies' in portfolio_summary:
    print(f"📈 Portfolio Companies: {portfolio_summary['total_companies']}")
    print(f"📊 Average Score: {portfolio_summary['average_score']:.1f}/100")

print(f"⚠️ Total Alerts: {len(intelligence_system.alerts)}")

print(f"\n🌟 COURSE ACHIEVEMENTS")
print("="*40)
achievements = [
    "Graph Database Mastery",
    "Data Modeling & Import", 
    "NLP & Knowledge Extraction",
    "Advanced Graph Analytics",
    "GraphRAG Implementation",
    "Intelligent Agents"
]

for achievement in achievements:
    print(f"✅ {achievement}")

print(f"\n🚀 You're ready to build the future of graph-powered AI!")
print(f"📚 Continue learning: https://graphacademy.neo4j.com/")

# Optional cleanup
# driver.close()
print(f"\n✅ Session complete!")

In [ ]:
# Simple Performance Visualization

def create_simple_dashboard():
    """Create a simple dashboard showing system performance"""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Financial Intelligence Agent Dashboard', fontsize=16, fontweight='bold')
    
    # 1. Portfolio Scores
    if intelligence_system.portfolio:
        companies = []
        scores = []
        
        for ticker, info in intelligence_system.portfolio.items():
            companies.append(f"{info['company_name'][:10]}\\n({ticker})")
            scores.append(info['investment_score'])
        
        bars = axes[0, 0].bar(companies, scores, color=['green' if s >= 70 else 'orange' if s >= 50 else 'red' for s in scores])
        axes[0, 0].set_title('Portfolio Investment Scores')
        axes[0, 0].set_ylabel('Score (0-100)')
        axes[0, 0].set_ylim(0, 100)
        
        # Add score labels on bars
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 1,
                           f'{score:.1f}', ha='center', va='bottom', fontweight='bold')
    else:
        axes[0, 0].text(0.5, 0.5, 'No Portfolio Data', ha='center', va='center', transform=axes[0, 0].transAxes)
        axes[0, 0].set_title('Portfolio Investment Scores')
    
    # 2. Analysis Timeline
    if intelligence_system.analysis_history:
        timestamps = [datetime.fromisoformat(item['timestamp']) for item in intelligence_system.analysis_history]
        cumulative_count = list(range(1, len(timestamps) + 1))
        
        axes[0, 1].plot(timestamps, cumulative_count, marker='o', linewidth=2, markersize=4)
        axes[0, 1].set_title('Analysis Activity Timeline')
        axes[0, 1].set_ylabel('Cumulative Analyses')
        axes[0, 1].tick_params(axis='x', rotation=45)
    else:
        axes[0, 1].text(0.5, 0.5, 'No Analysis History', ha='center', va='center', transform=axes[0, 1].transAxes)
        axes[0, 1].set_title('Analysis Activity Timeline')
    
    # 3. Tool Usage
    tool_usage = {}
    for tool_name, tool in integrated_agent.tools.items():
        if hasattr(tool, 'call_count'):
            tool_usage[tool_name.replace('_', ' ').title()] = tool.call_count
    
    if tool_usage:
        tools = list(tool_usage.keys())
        counts = list(tool_usage.values())
        
        axes[1, 0].barh(tools, counts, color='lightblue')
        axes[1, 0].set_title('Tool Usage Statistics')
        axes[1, 0].set_xlabel('Number of Calls')
        
        # Add count labels
        for i, count in enumerate(counts):
            axes[1, 0].text(count + 0.1, i, str(count), va='center')
    else:
        axes[1, 0].text(0.5, 0.5, 'No Tool Usage Data', ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Tool Usage Statistics')
    
    # 4. Alert Distribution
    if intelligence_system.alerts:
        alert_types = {}
        for alert in intelligence_system.alerts:
            alert_type = alert['type']
            alert_types[alert_type] = alert_types.get(alert_type, 0) + 1
        
        colors = ['red', 'orange', 'yellow', 'blue', 'green']
        axes[1, 1].pie(alert_types.values(), labels=alert_types.keys(), autopct='%1.0f',
                      colors=colors[:len(alert_types)])
        axes[1, 1].set_title('Alert Distribution')
    else:
        axes[1, 1].text(0.5, 0.5, 'No Alerts Generated', ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Alert Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\\n📊 DASHBOARD SUMMARY")
    print("="*50)
    
    if intelligence_system.portfolio:
        summary = intelligence_system.get_portfolio_summary()
        print(f"📈 Portfolio: {summary['total_companies']} companies, avg score {summary['average_score']:.1f}")
    else:
        print("📈 Portfolio: Empty")
    
    agent_status = integrated_agent.get_status()
    print(f"🤖 Agent: {agent_status['total_queries']} queries, {agent_status['entities_tracked']} entities tracked")
    print(f"📊 System: {len(intelligence_system.analysis_history)} analyses, {len(intelligence_system.alerts)} alerts")
    
    if tool_usage:
        total_tool_calls = sum(tool_usage.values())
        print(f"🔧 Tools: {total_tool_calls} total calls across {len(tool_usage)} tools")

# Create the dashboard
print("=== FINANCIAL INTELLIGENCE DASHBOARD ===")
create_simple_dashboard()

In [ ]:
# Practice Exercises

print("=== PRACTICE EXERCISES ===\\n")

print("🎯 Try these exercises to explore the financial intelligence agent system:\\n")

exercises = [
    {
        'title': '1. Analyze a New Company',
        'description': 'Add a new company to your graph database and analyze it',
        'example': '''
# First, add a new company to the database
run_query(\"\"\"
CREATE (c:Company {
    name: 'Tesla Inc',
    ticker: 'TSLA', 
    sector: 'Automotive',
    market_cap: 800000000000
})
\"\"\")

# Then analyze it
analysis = integrated_agent.full_investment_analysis("Tesla Inc", "TSLA")
print(f"Investment Score: {analysis['investment_score']:.1f}/100")
        '''
    },
    {
        'title': '2. Create Custom Risk Factors',
        'description': 'Add new risk factors and connect them to companies',
        'example': '''
# Add a new risk factor
run_query(\"\"\"
CREATE (r:RiskFactor {name: 'Climate Change Risk', severity: 'High'})
\"\"\")

# Connect it to a company
run_query(\"\"\"
MATCH (c:Company {name: 'Tesla Inc'}), (r:RiskFactor {name: 'Climate Change Risk'})
CREATE (c)-[:FACES_RISK]->(r)
\"\"\")

# Analyze the updated risk profile
risk_analysis = integrated_agent.tools['graph_reasoning'].execute({
    'analysis_type': 'risk_network',
    'company_name': 'Tesla Inc'
})
print(f"Risk Level: {risk_analysis['risk_level']}")
        '''
    },
    {
        'title': '3. Portfolio Monitoring',
        'description': 'Set up automated portfolio monitoring with alerts',
        'example': '''
# Add multiple companies to monitor
companies = [("Tesla Inc", "TSLA"), ("Google Inc", "GOOGL")]

for company, ticker in companies:
    intelligence_system.add_to_portfolio(company, ticker)

# Check for alerts
alerts = intelligence_system.get_alerts(severity='high')
for alert in alerts:
    print(f"⚠️ {alert['message']}")

# Generate monitoring report
report = intelligence_system.generate_portfolio_report()
print(report)
        '''
    },
    {
        'title': '4. Custom Analysis Tool',
        'description': 'Create your own analysis tool and add it to the agent',
        'example': '''
class ESGAnalysisTool(SimpleTool):
    def __init__(self):
        super().__init__("ESG Analysis", "Analyze Environmental, Social, Governance factors")
    
    def execute(self, parameters):
        company_name = parameters.get('company_name')
        # Mock ESG scoring logic
        esg_score = random.randint(60, 95)
        return {
            'success': True,
            'company': company_name,
            'esg_score': esg_score,
            'rating': 'A' if esg_score > 90 else 'B' if esg_score > 80 else 'C'
        }

# Add to agent
integrated_agent.tools['esg_analysis'] = ESGAnalysisTool()

# Use the new tool
esg_result = integrated_agent.tools['esg_analysis'].execute({'company_name': 'Apple Inc'})
print(f"ESG Score: {esg_result['esg_score']}, Rating: {esg_result['rating']}")
        '''
    },
    {
        'title': '5. Competitive Intelligence',
        'description': 'Analyze competitive relationships and market positioning',
        'example': '''
# Add competitive relationships
run_query(\"\"\"
MATCH (apple:Company {name: 'Apple Inc'}), (google:Company {name: 'Google Inc'})
CREATE (apple)-[:COMPETES_WITH]->(google)
CREATE (google)-[:COMPETES_WITH]->(apple)
\"\"\")

# Analyze competitive landscape
competitive_analysis = integrated_agent.tools['graph_reasoning'].execute({
    'analysis_type': 'competitive',
    'company_name': 'Apple Inc'
})

print(f"Competitive Intensity: {competitive_analysis['competitive_intensity']}")
print(f"Direct Competitors: {competitive_analysis['direct_competitors']}")
        '''
    }
]

for exercise in exercises:
    print(f"\\n{exercise['title']}")
    print(f"   📝 {exercise['description']}")
    print(f"   💻 Example Code:")
    
    # Print code with proper indentation
    for line in exercise['example'].strip().split('\\n'):
        print(f"   {line}")

print(f"\\n\\n🚀 EXTENSION IDEAS:")

extensions = [
    "🔗 Connect to real financial APIs (Alpha Vantage, Yahoo Finance, SEC EDGAR)",
    "🤖 Integrate with real LLM APIs (OpenAI, Anthropic, Google)",
    "📊 Build interactive web dashboard with Streamlit or Dash", 
    "📧 Add email alerts and automated reporting",
    "🔄 Implement real-time data streaming and updates",
    "🧠 Add machine learning models for price prediction",
    "📱 Create mobile app interface for portfolio monitoring",
    "🔐 Add user authentication and multi-tenant support"
]

for extension in extensions:
    print(f"  {extension}")

print(f"\\n💡 Remember: The goal is to understand how agents can reason over graph data!")
print(f"Start simple, then gradually add complexity as you master each concept.")

In [ ]:
## Module Summary and Course Completion

🎉 **Congratulations!** You have successfully completed Module 6: Agents and the entire **Modular GenAI Workshops 2025** course!

### What You've Accomplished in This Module

✅ **Simple Agent Architecture** - Built financial agents with memory and basic tool integration

✅ **Graph-Based Reasoning** - Implemented reasoning patterns that leverage graph relationships for competitive analysis and risk assessment

✅ **Tool Integration** - Connected agents to external data sources like market data and news analysis

✅ **Complete Intelligence System** - Created a comprehensive portfolio monitoring system with alerts and reporting

### Key Technical Skills Gained

- **Agent Memory Systems**: Conversation tracking and entity monitoring
- **Tool-Based Architecture**: Modular capabilities that agents can use dynamically  
- **Graph Reasoning**: Multi-hop analysis over connected financial data
- **Data Integration**: Combining multiple data sources for comprehensive analysis
- **System Monitoring**: Performance tracking and alert generation

### Complete Course Journey

Throughout this course, you've mastered:

1. **📊 Graph Basics** - Neo4j fundamentals and Cypher query language
2. **🏗️ Structured Data** - Data modeling, import strategies, and relationship design  
3. **📝 Unstructured Data** - NLP, entity extraction, and knowledge graph construction
4. **🔍 Graph Analytics** - Advanced algorithms, centrality measures, and business insights
5. **🔗 Retrievers** - Sophisticated RAG implementations with graph-enhanced retrieval
6. **🤖 Agents** - Intelligent agents with graph reasoning and tool integration

### Business Applications Enabled

Your new skills enable you to build:
- **💼 Investment Analysis Systems** - AI agents that can analyze companies and make recommendations
- **⚠️ Risk Management Tools** - Systems that identify and monitor financial risks across networks
- **📊 Portfolio Intelligence** - Automated portfolio monitoring with intelligent alerts
- **🏢 Corporate Intelligence** - Competitive analysis and market positioning tools
- **🔍 Decision Support Systems** - AI assistants that provide explainable, evidence-based recommendations

### What Makes This Approach Powerful

- **🕸️ Graph-Enhanced Intelligence**: Agents can reason over complex relationships, not just isolated data points
- **🔄 Dynamic Tool Usage**: Agents adapt their analysis approach based on available data and tools
- **💭 Explainable Decisions**: Graph-based reasoning provides clear audit trails for AI decisions
- **📈 Scalable Architecture**: System grows with your data and analysis needs

### Next Steps for Continued Growth

1. **🚀 Apply to Your Domain**: Adapt these patterns to your specific business use cases
2. **🔬 Explore Advanced Features**: Investigate Neo4j Graph Data Science and advanced AI capabilities  
3. **👥 Join the Community**: Connect with Neo4j and AI communities for ongoing learning
4. **🏆 Earn Certifications**: Validate your skills with Neo4j and AI certifications
5. **📢 Share Your Success**: Contribute your innovations back to the community

### Learning Resources for Your Journey

- **📚 [Neo4j Documentation](https://neo4j.com/docs/)** - Comprehensive technical references
- **🎓 [GraphAcademy](https://graphacademy.neo4j.com/)** - Advanced courses and certifications
- **💬 [Neo4j Community](https://community.neo4j.com/)** - Forums and expert discussions
- **🔬 [Graph Data Science](https://neo4j.com/docs/graph-data-science/)** - Advanced analytics library
- **🏗️ [LangChain/LangGraph](https://python.langchain.com/)** - Agent development frameworks

### Final System Status

Your financial intelligence system is now capable of:
- ✅ Analyzing companies using multiple data sources
- ✅ Reasoning over graph relationships for deeper insights  
- ✅ Managing investment portfolios with intelligent monitoring
- ✅ Generating alerts and automated reports
- ✅ Learning and adapting from each analysis

**Thank you for your dedication to mastering graph-powered AI!** 🙏

You're now equipped to build the next generation of intelligent, explainable, and powerful AI systems that leverage the full potential of connected data. Go forth and build amazing graph-powered AI systems! 🚀

---

*Modular GenAI Workshops 2025 - Building the Future of Graph-Powered Intelligence*

# Final System Cleanup and Summary

print("🎊 MODULE 6: AGENTS - COMPLETED SUCCESSFULLY! 🎊")
print("🏆 MODULAR GENAI WORKSHOPS 2025 - COURSE COMPLETED! 🏆\\n")

# Final system statistics
print("📊 FINAL SYSTEM STATISTICS")
print("="*50)

# Agent performance
agent_status = integrated_agent.get_status()
print(f"🤖 Agent Performance:")
print(f"   Name: {agent_status['name']}")
print(f"   Total Queries: {agent_status['total_queries']}")
print(f"   Conversations: {agent_status['conversations_remembered']}")
print(f"   Entities Tracked: {agent_status['entities_tracked']}")
print(f"   Available Tools: {agent_status['available_tools']}")

# System performance
print(f"\\n🏢 Portfolio Management:")
if intelligence_system.portfolio:
    summary = intelligence_system.get_portfolio_summary()
    print(f"   Companies: {summary['total_companies']}")
    print(f"   Average Score: {summary['average_score']:.1f}/100")
    print(f"   Best Performer: {summary['companies'][0]['company']} ({summary['companies'][0]['score']:.1f})")
else:
    print("   No portfolio created in this session")

print(f"\\n📈 Analysis Activity:")
print(f"   Total Analyses: {len(intelligence_system.analysis_history)}")
print(f"   Alerts Generated: {len(intelligence_system.alerts)}")

# Tool usage summary
print(f"\\n🔧 Tool Usage Summary:")
total_tool_calls = 0
for tool_name, tool in integrated_agent.tools.items():
    if hasattr(tool, 'call_count'):
        print(f"   {tool_name.replace('_', ' ').title()}: {tool.call_count} calls")
        total_tool_calls += tool.call_count

print(f"   Total Tool Calls: {total_tool_calls}")

# Course completion summary
print(f"\\n🎓 COURSE COMPLETION SUMMARY")
print("="*50)

course_achievements = [
    "✅ Graph Database Fundamentals Mastered",
    "✅ Data Modeling and Import Strategies Learned", 
    "✅ NLP and Knowledge Extraction Implemented",
    "✅ Advanced Graph Analytics Applied",
    "✅ GraphRAG Systems Built",
    "✅ Intelligent Agents Developed",
    "✅ Production-Ready Systems Created"
]

for achievement in course_achievements:
    print(f"   {achievement}")

print(f"\\n🌟 Key Technical Capabilities Acquired:")

capabilities = [
    "Building graph-enhanced AI agents with reasoning capabilities",
    "Implementing tool-based architectures for dynamic analysis",
    "Creating financial intelligence systems with monitoring",
    "Designing explainable AI with graph-based evidence trails",
    "Integrating multiple data sources for comprehensive analysis"
]

for capability in capabilities:
    print(f"   • {capability}")

print(f"\\n🚀 You're now ready to:")
print(f"   • Build sophisticated financial intelligence applications")
print(f"   • Create AI agents that can reason over complex connected data")
print(f"   • Implement production-ready graph-powered AI systems")
print(f"   • Design explainable and auditable AI decision systems")

# Optional cleanup
print(f"\\n🧹 Cleaning up resources...")
# Uncomment the next line to close database connection
# driver.close()
print("✅ Session completed successfully")

print(f"\\n📚 Continue Learning:")
print(f"   • Neo4j Community: https://community.neo4j.com/")
print(f"   • Graph Data Science: https://neo4j.com/docs/graph-data-science/")
print(f"   • GraphAcademy: https://graphacademy.neo4j.com/")

print(f"\\n💫 Thank you for completing the Modular GenAI Workshops 2025!")
print(f"🎉 You've built the foundation for the future of graph-powered AI!")

In [ ]:
# Comprehensive agent evaluation and dashboard
def create_agent_performance_dashboard():
    """Create a comprehensive performance dashboard for the agent system"""
    
    # Collect performance metrics
    system_status = coordinator.get_system_status()
    
    # Create visualizations
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Agent Activity Levels
    agent_names = list(system_status['agent_details'].keys())
    agent_actions = [details['status']['total_actions'] for details in system_status['agent_details'].values()]
    
    axes[0, 0].bar(agent_names, agent_actions)
    axes[0, 0].set_title('Agent Activity Levels')
    axes[0, 0].set_ylabel('Total Actions')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # 2. Memory Usage
    memory_items = [details['status']['memory_items'] for details in system_status['agent_details'].values()]
    
    axes[0, 1].bar(agent_names, memory_items, color='orange')
    axes[0, 1].set_title('Agent Memory Usage')
    axes[0, 1].set_ylabel('Memory Items')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Task Completion Timeline
    if coordinator.collaboration_history:
        task_times = [datetime.fromisoformat(h['timestamp']) for h in coordinator.collaboration_history]
        task_counts = list(range(1, len(task_times) + 1))
        
        axes[0, 2].plot(task_times, task_counts, marker='o')
        axes[0, 2].set_title('Task Completion Timeline')
        axes[0, 2].set_ylabel('Cumulative Tasks')
        axes[0, 2].tick_params(axis='x', rotation=45)
    else:
        axes[0, 2].text(0.5, 0.5, 'No task history', ha='center', va='center', transform=axes[0, 2].transAxes)
        axes[0, 2].set_title('Task Completion Timeline')
    
    # 4. Specialization Distribution
    all_specialties = []
    for details in system_status['agent_details'].values():
        all_specialties.extend(details['specialties'])
    
    specialty_counts = pd.Series(all_specialties).value_counts()
    
    axes[1, 0].pie(specialty_counts.values, labels=specialty_counts.index, autopct='%1.1f%%')
    axes[1, 0].set_title('Agent Specialization Distribution')
    
    # 5. Tool Usage
    all_tools = []
    for details in system_status['agent_details'].values():
        all_tools.extend(details['status']['available_tools'])
    
    tool_counts = pd.Series(all_tools).value_counts()
    
    axes[1, 1].barh(range(len(tool_counts)), tool_counts.values)
    axes[1, 1].set_yticks(range(len(tool_counts)))
    axes[1, 1].set_yticklabels(tool_counts.index)
    axes[1, 1].set_title('Tool Availability Across Agents')
    axes[1, 1].set_xlabel('Number of Agents with Tool')
    
    # 6. System Health Summary
    health_metrics = {
        'Active Agents': system_status['available_agents'],
        'Total Tasks': system_status['completed_tasks'],
        'Shared Memory': system_status['shared_memory_items'],
        'Avg Actions/Agent': np.mean(agent_actions) if agent_actions else 0
    }
    
    y_pos = np.arange(len(health_metrics))
    values = list(health_metrics.values())
    
    axes[1, 2].barh(y_pos, values, color='green', alpha=0.7)
    axes[1, 2].set_yticks(y_pos)
    axes[1, 2].set_yticklabels(health_metrics.keys())
    axes[1, 2].set_title('System Health Metrics')
    axes[1, 2].set_xlabel('Value')
    
    plt.tight_layout()
    plt.show()
    
    return health_metrics

# Create dashboard
print("=== AGENT PERFORMANCE DASHBOARD ===")
health_metrics = create_agent_performance_dashboard()

print("\nSystem Health Summary:")
for metric, value in health_metrics.items():
    print(f"- {metric}: {value:.2f}")

In [ ]:
# Final comprehensive test scenario
def run_comprehensive_test_scenario():
    """Run a comprehensive test scenario simulating real-world usage"""
    
    print("=== COMPREHENSIVE TEST SCENARIO ===")
    print("Simulating a day in the life of the financial analysis agent system\n")
    
    # Scenario: Security alert received
    print("🚨 SCENARIO 1: Security Alert Received")
    print("Alert: Suspicious transaction patterns detected")
    
    # Step 1: Initial monitoring check
    monitoring_result = coordinator.assign_task(
        "Perform immediate system monitoring check for suspicious activities",
        'monitoring'
    )
    
    if monitoring_result['success']:
        print(f"✅ Monitoring completed by {monitoring_result['assigned_agent']}")
        print(f"   Response: {monitoring_result['result']['final_response'][:100]}...")
    
    # Step 2: Detailed risk analysis
    print("\n📊 SCENARIO 2: Detailed Risk Analysis")
    risk_analysis_result = coordinator.assign_task(
        "Analyze risk profiles of all accounts and identify high-risk patterns",
        'risk_analysis'
    )
    
    if risk_analysis_result['success']:
        print(f"✅ Risk analysis completed by {risk_analysis_result['assigned_agent']}")
        print(f"   Key findings: {risk_analysis_result['result']['final_response'][:150]}...")
    
    # Step 3: Pattern analysis
    print("\n🔍 SCENARIO 3: Advanced Pattern Analysis")
    pattern_result = coordinator.assign_task(
        "Identify unusual transaction patterns and network anomalies",
        'pattern_analysis'
    )
    
    if pattern_result['success']:
        print(f"✅ Pattern analysis completed by {pattern_result['assigned_agent']}")
        print(f"   Patterns found: {pattern_result['result']['final_response'][:150]}...")
    
    # Step 4: Collaborative comprehensive analysis
    print("\n🤝 SCENARIO 4: Collaborative Investigation")
    collaborative_result = coordinator.collaborative_analysis(
        "Conduct comprehensive investigation of potential fraud network involving multiple accounts and suspicious patterns"
    )
    
    print(f"✅ Collaborative analysis completed")
    print(f"   Subtasks: {len(collaborative_result['subtasks'])}")
    print(f"   Agents involved: {len(set(r['result']['assigned_agent'] for r in collaborative_result['subtask_results'] if r['result']['success']))}")
    print(f"   Final synthesis: {collaborative_result['synthesis'][:200]}...")
    
    # Step 5: Generate executive summary
    print("\n📋 SCENARIO 5: Executive Summary Generation")
    summary_task = "Generate executive summary of investigation findings with actionable recommendations"
    summary_result = coordinator.assign_task(summary_task, 'comprehensive_analysis')
    
    if summary_result['success']:
        print(f"✅ Executive summary generated by {summary_result['assigned_agent']}")
        print(f"   Summary: {summary_result['result']['final_response'][:200]}...")
    
    # Performance metrics
    print("\n📈 PERFORMANCE METRICS")
    final_status = coordinator.get_system_status()
    
    total_actions = sum(details['status']['total_actions'] for details in final_status['agent_details'].values())
    total_memory = sum(details['status']['memory_items'] for details in final_status['agent_details'].values())
    
    print(f"Total Tasks Completed: {final_status['completed_tasks']}")
    print(f"Total Agent Actions: {total_actions}")
    print(f"Total Memory Items: {total_memory}")
    print(f"Agents Utilized: {final_status['total_agents']}")
    
    # Success rate calculation
    successful_tasks = sum(1 for h in coordinator.collaboration_history if 'error' not in h['result'])
    success_rate = (successful_tasks / len(coordinator.collaboration_history)) * 100 if coordinator.collaboration_history else 0
    
    print(f"Task Success Rate: {success_rate:.1f}%")
    
    return {
        'total_tasks': final_status['completed_tasks'],
        'total_actions': total_actions,
        'success_rate': success_rate,
        'agents_used': final_status['total_agents']
    }

# Run the comprehensive test
test_results = run_comprehensive_test_scenario()

print("\n" + "="*60)
print("COMPREHENSIVE TEST COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"Final Results: {test_results}")

## Module Summary and Course Completion

🎉 **Congratulations!** You have successfully completed Module 6: Agents and the entire Modular GenAI Workshops 2025 course!

### What You've Accomplished

In this final module, you learned to:
- **Design sophisticated agent architectures** that leverage graph knowledge for intelligent reasoning
- **Implement multi-hop reasoning patterns** over complex graph structures
- **Build dynamic tool integration systems** that can generate and execute queries safely
- **Create multi-agent coordination systems** for collaborative problem-solving
- **Develop adaptive learning mechanisms** for continuous improvement
- **Build comprehensive monitoring and alerting systems** for real-time analysis

### Complete Course Journey

Throughout this course, you've mastered:

1. **Module 1: Graph Basics** - Foundation of graph databases and Cypher
2. **Module 2: Structured Data** - Data modeling and import strategies
3. **Module 3: Unstructured Data** - NLP and knowledge extraction
4. **Module 4: Graph Analytics** - Advanced algorithms and business insights
5. **Module 5: Retrievers** - Sophisticated RAG implementation patterns
6. **Module 6: Agents** - Intelligent agents with graph reasoning

### Key Technical Achievements

- **Graph-Enhanced AI**: Successfully combined graph databases with AI/ML workflows
- **Production-Ready Systems**: Built scalable, monitored, and maintainable solutions
- **Multi-Modal Integration**: Connected structured data, unstructured text, and intelligent reasoning
- **Enterprise Patterns**: Implemented patterns suitable for real-world business applications

### Business Impact

The skills you've developed enable:
- **Fraud Detection**: Advanced pattern recognition and risk assessment
- **Knowledge Management**: Intelligent document retrieval and analysis
- **Decision Support**: AI agents that can reason over complex business data
- **Process Automation**: Intelligent automation with explainable reasoning

### Next Steps

Now that you've completed the course:

1. **Apply to Your Domain**: Adapt these patterns to your specific business use cases
2. **Explore Advanced Features**: Investigate cutting-edge Neo4j and AI capabilities
3. **Join the Community**: Connect with the Neo4j community for continued learning
4. **Consider Certification**: Validate your skills with Neo4j certification programs
5. **Share Your Success**: Contribute to the community with your experiences and innovations

### Resources for Continued Learning

- [Neo4j Documentation](https://neo4j.com/docs/)
- [Neo4j Community Forum](https://community.neo4j.com/)
- [Graph Data Science Library](https://neo4j.com/docs/graph-data-science/)
- [Neo4j Certification](https://neo4j.com/graphacademy/)
- [Advanced Workshops and Training](https://neo4j.com/training/)

Thank you for your dedication to learning these advanced graph-based AI techniques. You're now equipped to build the next generation of intelligent, graph-powered applications!

In [ ]:
# Final celebration and cleanup
print("🎊 MODULE 6: AGENTS - COMPLETED SUCCESSFULLY! 🎊")
print("\n🏆 MODULAR GENAI WORKSHOPS 2025 - COURSE COMPLETED! 🏆")

print("\n" + "="*70)
print("                    FINAL COURSE STATISTICS")
print("="*70)

# Calculate final statistics
final_system_status = coordinator.get_system_status()
total_agent_actions = sum(details['status']['total_actions'] for details in final_system_status['agent_details'].values())
total_memory_items = sum(details['status']['memory_items'] for details in final_system_status['agent_details'].values())

course_stats = {
    'Modules Completed': 6,
    'Agents Created': final_system_status['total_agents'],
    'Total Agent Actions': total_agent_actions,
    'Collaborative Tasks': final_system_status['completed_tasks'],
    'Memory Items Generated': total_memory_items,
    'Tools Implemented': 5,  # CypherQuery, GraphAnalytics, DynamicQuery, Monitoring, plus base tools
    'Reasoning Chains Created': sum(len(agent_info['agent'].reasoning_chain) for agent_info in coordinator.agents.values()),
    'Multi-Agent Collaborations': len([h for h in coordinator.collaboration_history if 'collaborative' in str(h)])
}

for stat, value in course_stats.items():
    print(f"{stat:<30}: {value:>10}")

print("\n🌟 Key Achievements Unlocked:")
achievements = [
    "✅ Graph Database Mastery",
    "✅ Advanced Cypher Proficiency", 
    "✅ Knowledge Graph Construction",
    "✅ Graph Analytics Implementation",
    "✅ GraphRAG System Development",
    "✅ Intelligent Agent Architecture",
    "✅ Multi-Agent Coordination",
    "✅ Production-Ready AI Systems"
]

for achievement in achievements:
    print(f"    {achievement}")

print("\n🚀 You're now ready to build the future of graph-powered AI!")
print("\n💡 Remember: The journey of learning never ends. Keep exploring, building, and sharing!")

# Optional cleanup
print("\n🧹 Cleaning up resources...")
# driver.close()  # Uncomment to close database connection
print("✅ Cleanup completed.")

print("\n🎓 Thank you for completing the Modular GenAI Workshops 2025!")
print("📧 Don't forget to share your success and join the Neo4j community!")